## 数据搜集
- 合并所有评分卡数据，并人工核对

In [3]:
import pandas as pd
import os
import numpy as np
import re
from funcs import *
import datetime
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

In [138]:
#合并所有公司评分卡内容
files = [f for f in files if '评分表' in f]
df = pd.DataFrame()
for f in files:
    company = f.split('/')[-2]
    d =pd.read_excel(f)
    df_s= d.dropna(subset=[u'序号'])[[u'实际情况']]
    df_s = df_s.rename(columns={u'实际情况':company})
    df = pd.concat([df,df_s],axis=1)
df[u'考核内容']= d.dropna(subset=[u'序号'])[[u'考核内容']]
df

#保存，由于评分卡不统一，需要人工核查评分卡对应内容内容
df.dropna(how='all').set_index(u'考核内容').to_csv('./features.csv',encoding ='utf-8')

,无锡国友塑料制品有限公司,砀山县雁发塑胶制品有限公司,宜兴市海燕塑料制品有限公司,佛山市南海区狮山联生塑料五金厂,深圳市南璋颜料有限公司,佛山市南海辉煌塑料有限公司,中山市新展塑料制品有限公司,四会市塑利莱科技有限公司,中山市威塑建材有限公司,惠州俊儿塑料科技有限公司,...,佛山市南海区狮山润泉实业有限公司,东莞市丰年塑胶制品有限公司,佛山市顺元塑胶制品有限公司,广州佳昊实业有限公司,中山南荣塑胶机械工业有限公司,阳江市顺和工业有限公司,东莞市佳成塑胶科技有限公司,太仓京润塑料制品有限公司,考核内容,序号
0,6,15,5,5,11.5,10,4,10,13,2,...,16,5,4,3,16,19,2,10,注册时间（年）,1
4,200,500,280,30,50,188.7,30,101,50,750,...,150,50,100,100,3948,1000,200,330.796,实收资本（万）,2
8,A,A,A,B,A,A,A,A,B,B,...,A,B,A,B,A,B,A,A,A类(产业园)；B类（非产业园）,3
10,no,no,no,no,no,no,no,no,no,no,...,no,否,no,no,no,否,否,no,是否有实控人变更、是否存在减资。（是/否）,4
12,no,no,no,no,no,no,no,no,no,no,...,yes,否,no,no,no,否,否,no,分支机构（是/否）,5
14,30,100,80,80,40,30,50,100,10,100,...,50,10,40,100,50,300,160,50,员工人数,6
17,租赁2500,自有8000,租赁9000,租赁42600,租赁2000,租赁4000,自有3000,租赁30000,租赁1000,租赁80000,...,租赁3000,租赁1000,租赁6000,租赁8000,租赁40000,自有38000,自有18000,租赁3000,厂房自有或是租赁㎡,7
21,no,yes,yes,yes,no,yes,无,yes,yes,no,...,yes,无,no,无,yes,有,无,no,商标专利,8
23,47,14,67,91,0.88,91,0.9,0.86,0.15,0.24,...,90,71,77,84,0.33,25,90,50.81,资产负债率%（1）,9
25,121,F,107,101,0.98,69,1.09,F,6.39,3.94,...,99,NaN,109,106,2.36,NaN,NaN,156.19,流动比率%（1）,10


### 数据核查
- 注册资本核查-
- 是否有实控人变更、是否存在减资。
- 法律诉讼核查  
- 撮合业务数核对
- 注册时间核对
- 企业征信数据核对  --核对征信报告数据
- 逾期历史数据核对
- 下游客户信息补充--核查平台信息

In [235]:
#核对后的数据
data = pd.read_excel('./data/A_scores_check.xlsx',header=0,index_col=0)
data = data.T

#核对信息,平台信息
d1 = pd.read_excel('./data/consumer_info.xlsx',header=1,encoding='utf-8')
tmp = d1[[u'下游客户情况',u'主营品类']]
d2 = data.merge(tmp,left_index=True,right_index=True,how='left')
d2.to_excel('./data/featurs_v2.xlsx')
d2.head()

,注册时间（年）,实收资本（万）,A类(产业园)；B类（非产业园）,是否有实控人变更、是否存在减资。（是/否）,分支机构（是/否）,员工人数,厂房自有或是租赁㎡,商标专利,资产负债率%（1）,流动比率%（1）,...,交易量（单位：吨）,采购稳定性（交易量连续2个月月均下滑30%）,交易频率,领导人信用记（优秀、良好、差）,企业征信,涉诉信息,逾期历史（次数）,评分,下游客户情况,主营品类
无锡国友塑料制品有限公司,6,200,A,no,no,30,租赁2500,0,47,121,...,69,0,0,无,NaN,0,NaN,38,中小型客户,塑料桶
砀山县雁发塑胶制品有限公司,15,500,A,no,no,100,自有8000,1,14,NaN,...,262,no,4个月,无,正常,0,2,72,农药瓶，安徽省化工研究院 、合肥星宇化学,PE
宜兴市海燕塑料制品有限公司,5,280,A,no,no,80,租赁9000,1,67,107,...,687,no,近三月交易有断档,无,正常,0,7,68,出口,集装袋
佛山市南海区狮山联生塑料五金厂,5,30,B,no,no,80,租赁42600,1,91,101,...,298,no,有间断,无,正常,0,0,57,中小型客户,PE
深圳市南璋颜料有限公司,11.5,50,A,no,no,40,租赁2000,0,88,98,...,402.86,交易截止1月,交易截止1月,无,正常,0,6,47,电缆电线厂,pvc加工助剂及辅料


In [255]:
#核对逾期数据
# d3=pd.read_excel('./data/consumer_info.xlsx',sheetname=2)
# data = pd.read_excel('./data/featurs_v2.xlsx',header=0,index_col=0)
# data.merge(right=d3[u'合计'].reset_index(),left_index=True,right_on='index',how='left').to_excel('./data/feature_v3.xlsx',index=True)

,注册时间（年）,实收资本（万）,A类(产业园)；B类（非产业园）,注册资本变更,法人变更,分支机构（是/否）,员工人数,厂房自有或是租赁㎡,商标专利,资产负债率%（1）,...,采购稳定性（交易量连续2个月月均下滑30%）,交易频率,领导人信用记（优秀、良好、差）,企业征信,涉诉信息,逾期历史（次数）,评分,下游客户情况,index,合计
20,7.0,200.000,A,增资,no,no,30,租赁2500,0,47.00,...,0,0,无,正常,0,NaN,38.0,中小企业,无锡国友塑料制品有限公司,1.0
8,16.0,500.000,A,增资,no,no,100,自有8000,1,14.00,...,no,4个月,无,正常,0,2.0,72.0,中小企业,砀山县雁发塑胶制品有限公司,3.0
21,7.0,280.000,A,NaN,no,no,80,租赁9000,1,67.00,...,no,近三月交易有断档,无,正常,0,7.0,68.0,出口,宜兴市海燕塑料制品有限公司,7.0
11,5.5,30.000,B,增资,no,no,80,租赁42600,1,91.00,...,no,有间断,无,正常,0,0.0,57.0,中小企业,佛山市南海区狮山联生塑料五金厂,3.0
56,11.5,50.000,A,NaN,no,no,40,租赁2000,0,88.00,...,交易截止1月,交易截止1月,无,正常,0,6.0,47.0,中小企业,深圳市南璋颜料有限公司,6.0
25,12.0,183.700,A,增资,no,no,30,租赁4000,1,91.00,...,no,5个月,无,欠息,1,1.0,68.0,中小企业,佛山市南海辉煌塑料有限公司,1.0
37,4.0,30.000,A,NaN,no,no,50,自有3000,0,90.00,...,no,2月有断档,无,正常,0,7.0,68.0,中小企业,中山市新展塑料制品有限公司,7.0
60,10.0,101.000,A,NaN,no,no,100,租赁30000,1,86.00,...,交易截止3月,交易截止3月,无,正常,0,1.0,54.0,市政工程,四会市塑利莱科技有限公司,1.0
62,13.0,50.000,B,NaN,no,no,10,租赁1000,1,15.00,...,交易截止2月,交易截止2月,无,正常,0,NaN,37.5,中小企业,中山市威塑建材有限公司,0.0
31,2.0,750.000,B,NaN,no,no,100,租赁80000,0,24.00,...,no,5月断档,无,正常,0,6.0,48.5,中小企业,惠州俊儿塑料科技有限公司,6.0


### 数据标准化-基础特征统计
- 注册地址，分支机构，商标专利，采用onehot编码
- 财务状况统一用百分比。	
- 企业征信数据：
- 企业贷款三种情况：有贷款（企业征信：正常，欠息，关注，不良），无贷款，无征信报告
- 下游客户情况统一为：中小微企业，中小企业，自营，市政工程，大中企业，出口
- 归一化，onehot编码

In [4]:
data = pd.read_excel('./data/feature_checked.xlsx')
dic = {'no':0,u'无':0,'yes':1,u'有':1,u'否':0,u'是':1,1:1,0:0,'A':1,'B':0,'自有':1,'租赁':0}
dic2 = {u'出口':4,u'市政工程':4,u'大中企业':4,u'中小企业':2,u'中小微企业':2,u'自营':1}
dic3 = {u'正常':1,u'欠息':2,u'关注':3,u'不良':4}
data.head(3)

,注册时间,实收资本,注册地址,注册资本变更,法人变更,分支机构,员工人数,厂房面积,商标专利,资产负债率,...,累计交易单数,交易量,采购稳定性,交易频率,领导人信用记,企业征信,涉诉信息,逾期历史,评分,下游客户情况
无锡国友塑料制品有限公司,7.0,200.0,A,增资,no,no,30,租赁2500,0,47.0,...,3,69.0,0,0,无,NaN,0,1.0,38.0,中小企业
砀山县雁发塑胶制品有限公司,16.0,500.0,A,增资,no,no,100,自有8000,1,14.0,...,8,262.0,no,4个月,无,正常,0,3.0,72.0,中小企业
宜兴市海燕塑料制品有限公司,7.0,280.0,A,NaN,no,no,80,租赁9000,1,67.0,...,20,687.0,no,近三月交易有断档,无,正常,0,7.0,68.0,出口


In [5]:
#拆分
data['厂房面积']= data[u'厂房面积'].apply(lambda x:re.findall('(\d+)',x)[0])
data['场地归属'] = data[u'厂房面积'].apply(lambda x:'租赁' if u'租赁' in x else '自有')
data.head(2)

,注册时间,实收资本,注册地址,注册资本变更,法人变更,分支机构,员工人数,厂房面积,商标专利,资产负债率,...,采购稳定性,交易频率,领导人信用记,企业征信,涉诉信息,逾期历史,评分,下游客户情况,厂房面积,场地归属
无锡国友塑料制品有限公司,7.0,200.0,A,增资,no,no,30,租赁2500,0,47.0,...,0,0,无,NaN,0,1.0,38.0,中小企业,2500,租赁
砀山县雁发塑胶制品有限公司,16.0,500.0,A,增资,no,no,100,自有8000,1,14.0,...,no,4个月,无,正常,0,3.0,72.0,中小企业,8000,自有


In [20]:
# #离散特征处理:one-hot编码
# data_new = pd.DataFrame()
# data_new['注册地址'] = data[u'注册地址'].map(dic)
# data_new['注册资本变更'] = pd.get_dummies(data[u'注册资本变更'])
# data_new['法人变更'] = data[u'法人变更'].map(dic)
# data_new['分支机构'] = data[u'分支机构'].map(dic)
# data_new['商标专利'] = data[u'商标专利'].map(dic)
# data_new[['场地租赁','场地自有']] = pd.get_dummies(data['场地归属'])
# data_new['有无贷款'] = data[u'有无贷款'].map(dic)
# data_new[['征信不良','征信关注','征信欠息','征信正常']] = pd.get_dummies(data[u'企业征信'])
# data_new['涉诉信息'] = data[u'涉诉信息']
# data_new['下游客户情况'] = data[u'下游客户情况'].map(dic2)
# data_new

,注册地址,注册资本变更,法人变更,分支机构,商标专利,场地租赁,场地自有,有无贷款,征信不良,征信关注,征信欠息,征信正常,涉诉信息,下游客户情况
无锡国友塑料制品有限公司,1,1,0,0,0.0,1,0,0.0,0,0,0,0,0,2.0
砀山县雁发塑胶制品有限公司,1,1,0,0,1.0,0,1,1.0,0,0,0,1,0,2.0
宜兴市海燕塑料制品有限公司,1,0,0,0,1.0,1,0,1.0,0,0,0,1,0,4.0
佛山市南海区狮山联生塑料五金厂,0,1,0,0,1.0,1,0,1.0,0,0,0,1,0,2.0
深圳市南璋颜料有限公司,1,0,0,0,0.0,1,0,1.0,0,0,0,1,0,2.0
佛山市南海辉煌塑料有限公司,1,1,0,0,1.0,1,0,1.0,0,0,1,0,1,2.0
中山市新展塑料制品有限公司,1,0,0,0,0.0,0,1,1.0,0,0,0,1,0,2.0
四会市塑利莱科技有限公司,1,0,0,0,1.0,1,0,1.0,0,0,0,1,0,4.0
中山市威塑建材有限公司,0,0,0,0,1.0,1,0,0.0,0,0,0,0,0,2.0
惠州俊儿塑料科技有限公司,0,0,0,0,0.0,1,0,0.0,0,0,0,0,0,2.0


In [27]:
#离散特征直接编号
data_new = pd.DataFrame()
data_new['注册地址'] = data[u'注册地址'].map(dic)
data_new['注册资本变更'] = pd.get_dummies(data[u'注册资本变更'])
data_new['法人变更'] = data[u'法人变更'].map(dic)
data_new['分支机构'] = data[u'分支机构'].map(dic)
data_new['商标专利'] = data[u'商标专利'].map(dic)
data_new['场地归属'] = data['场地归属'].map(dic)
data_new['有无贷款'] = data[u'有无贷款'].map(dic)
data_new['企业征信'] = data[u'企业征信'].map(dic3)
data_new['涉诉信息'] = data[u'涉诉信息']
data_new['下游客户情况'] = data[u'下游客户情况'].map(dic2)
data_new

,注册地址,注册资本变更,法人变更,分支机构,商标专利,场地归属,有无贷款,企业征信,涉诉信息,下游客户情况
无锡国友塑料制品有限公司,1,1,0,0,0.0,0,0.0,NaN,0,2.0
砀山县雁发塑胶制品有限公司,1,1,0,0,1.0,1,1.0,1.0,0,2.0
宜兴市海燕塑料制品有限公司,1,0,0,0,1.0,0,1.0,1.0,0,4.0
佛山市南海区狮山联生塑料五金厂,0,1,0,0,1.0,0,1.0,1.0,0,2.0
深圳市南璋颜料有限公司,1,0,0,0,0.0,0,1.0,1.0,0,2.0
佛山市南海辉煌塑料有限公司,1,1,0,0,1.0,0,1.0,2.0,1,2.0
中山市新展塑料制品有限公司,1,0,0,0,0.0,1,1.0,1.0,0,2.0
四会市塑利莱科技有限公司,1,0,0,0,1.0,0,1.0,1.0,0,4.0
中山市威塑建材有限公司,0,0,0,0,1.0,0,0.0,NaN,0,2.0
惠州俊儿塑料科技有限公司,0,0,0,0,0.0,0,0.0,NaN,0,2.0


In [28]:
#不归一化
data_new['注册时间']=(data[u'注册时间'])
data_new['实收资本']= (data[u'实收资本'])
data_new['员工人数'] = (data[u'员工人数'])
tmp = (data['厂房面积'].astype(float))
data_new['厂房面积'] = tmp
data_new['资产负债率'] = (data[u'资产负债率'])
data_new['流动比率'] = (data[u'流动比率'])
data_new['净资产'] = (data[u'净资产'])
data_new['货币资金'] = (data[u'货币资金'])
data_new['存货周转天数'] = (data[u'存货周转天数'])
data_new['应收账款周转天数'] =(data[u'应收账款周转天数'])
data_new['主营业务利润率'] =(data[u'主营业务利润率'])
data_new['净资产增长率'] = (data[u'净资产增长率'])
data_new['销售增长率'] = (data[u'销售增长率'])
data_new['当地经济环境'] = (data[u'当地经济环境'])
data_new['撮合单数'] = (data[u'撮合单数'])
# data_new.to_csv('./data/features_card_v2.csv') #离散特征合并
data_new.to_csv('./data/features_card.csv') #离散特征合并
data_new

,注册地址,注册资本变更,法人变更,分支机构,商标专利,场地归属,有无贷款,企业征信,涉诉信息,下游客户情况,...,流动比率,净资产,货币资金,存货周转天数,应收账款周转天数,主营业务利润率,净资产增长率,销售增长率,当地经济环境,撮合单数
无锡国友塑料制品有限公司,1,1,0,0,0.0,0,0.0,NaN,0,2.0,...,121.00,204.0,9.0,25.00,201.00,12.00,284.00,NaN,7.50,0.0
砀山县雁发塑胶制品有限公司,1,1,0,0,1.0,1,1.0,1.0,0,2.0,...,NaN,1879.0,NaN,15.00,49.00,14.00,12.00,87.00,8.40,0.0
宜兴市海燕塑料制品有限公司,1,0,0,0,1.0,0,1.0,1.0,0,4.0,...,107.00,367.0,372.0,11.00,57.00,13.00,-2.00,-37.00,7.50,0.0
佛山市南海区狮山联生塑料五金厂,0,1,0,0,1.0,0,1.0,1.0,0,2.0,...,101.00,222.0,10.0,27.00,62.00,8.00,55.00,39.00,7.80,0.0
深圳市南璋颜料有限公司,1,0,0,0,0.0,0,1.0,1.0,0,2.0,...,98.00,208.0,616.0,4.00,74.00,12.00,13.00,-12.00,8.60,3.0
佛山市南海辉煌塑料有限公司,1,1,0,0,1.0,0,1.0,2.0,1,2.0,...,69.00,53.0,176.0,101.00,154.00,12.00,518.00,12.00,8.50,17.0
中山市新展塑料制品有限公司,1,0,0,0,0.0,1,1.0,1.0,0,2.0,...,109.00,71.0,7.0,0.00,113.00,9.00,82.00,237.00,7.10,20.0
四会市塑利莱科技有限公司,1,0,0,0,1.0,0,1.0,1.0,0,4.0,...,NaN,152.0,NaN,NaN,NaN,NaN,NaN,NaN,4.80,0.0
中山市威塑建材有限公司,0,0,0,0,1.0,0,0.0,NaN,0,2.0,...,639.00,56.0,52.0,38.00,0.00,17.00,1.00,-27.00,7.10,0.0
惠州俊儿塑料科技有限公司,0,0,0,0,0.0,0,0.0,NaN,0,2.0,...,394.00,749.0,3.0,686.00,106.00,1.00,NaN,NaN,7.10,0.0


In [30]:
data_new.describe().T

,count,mean,std,min,25%,50%,75%,max
注册地址,66.0,0.424242,0.498015,0.00,0.00,0.00,1.000,1.00
注册资本变更,66.0,0.227273,0.422282,0.00,0.00,0.00,0.000,1.00
法人变更,66.0,0.060606,0.240435,0.00,0.00,0.00,0.000,1.00
分支机构,66.0,0.121212,0.328875,0.00,0.00,0.00,0.000,1.00
商标专利,65.0,0.461538,0.502398,0.00,0.00,0.00,1.000,1.00
场地归属,66.0,0.303030,0.463090,0.00,0.00,0.00,1.000,1.00
有无贷款,58.0,0.706897,0.459161,0.00,0.00,1.00,1.000,1.00
企业征信,41.0,1.341463,0.728346,1.00,1.00,1.00,1.000,4.00
涉诉信息,66.0,0.363636,0.715464,0.00,0.00,0.00,1.000,4.00
下游客户情况,53.0,2.490566,0.932792,1.00,2.00,2.00,4.000,4.00


### 交易数据特征-以客户为样本
- 合作时间，订单总数，交易总量，平均交易量（吨），平均交易额（万），最高交易额，最高交易量，交易时间间隔均值，
- 合作供应商数量，购买产品类数
- 金融服务（贷款订单）统计：成交量，成交额，订单数,合作时间。

In [17]:
indexs = ['订单总数','合作时间（月）','最近交易（X月之前）','最近连续交易月数','订单时间间隔均值（天）','订单时间间隔方差（天）',
          '交易总量','交易量最大值','交易量最小值','交易量方差','交易量均值','交易额总量','交易额最大值','交易额最小值','交易额方差',
          '交易额均值','交易稳定性（近三个月交易量变化均值）','供应商数量','采购产品种类数量',
          '账期订单占比','账期订单交易量均值','账期订单交易额均值','账期合作时间（月）','最近一笔账期订单（X月之前）']
data_add = pd.DataFrame(index=indexs)
data_add.shape

(24, 0)

In [18]:
#合并所有公司新增特征
t_now =datetime.datetime(2017,7,28)
#读取数据文件夹下所有文件名
files = VisitDir('./data/client_data/')
files =[f for f in files if '交易记录' in f]
for f in files:
    col = f.split('/')[-2]
    feature = []
    trade = pd.read_excel(f)
    trade =trade.dropna(thresh=5)
    num = trade[u'创建时间'].shape[0]
    feature.append(num)
    feature.append(coo_months(trade[u'创建时间']))
    feature.append(recent_month(trade[u'创建时间'],t_now))
    feature.append(recent_last_month(trade[u'创建时间']))
    
    tim_st = time_gap(trade[u'创建时间'])
    feature.append(tim_st[0]) #均值
    feature.append(tim_st[1]) #方差
    
    t_num = trade[u'销售数量（吨）'].agg(['sum','max','min','std','mean'])
    t_money = trade[u'销售金额（元）'].agg(['sum','max','min','std','mean'])
    feature.extend(t_num)
    feature.extend(t_money)
    feature.append(trade_stab(trade))
    feature.append(trade[u'货物'].apply(lambda x:x.split(' ')[1]).value_counts().shape[0])
    feature.append(trade[u'货物'].apply(lambda x:x.split(' ')[0]).value_counts().shape[0])
    
    zhangqi = trade[trade[u'是否账期']==u'是']
    if zhangqi.shape[0]>0:
        feature.append(zhangqi.shape[0]*1.0/num)
        feature.append(zhangqi[u'销售数量（吨）'].mean())
        feature.append(zhangqi[u'销售金额（元）'].mean())
        feature.append(coo_months(zhangqi[u'创建时间']))
        feature.append(recent_month(zhangqi[u'创建时间'],t_now))
    else:
        feature.append(np.nan)
        feature.append(np.nan)
        feature.append(np.nan)
        feature.append(np.nan)
        feature.append(np.nan)
    data_add[col] = feature
data_add = data_add.T
data_add

,订单总数,合作时间（月）,最近交易（X月之前）,最近连续交易月数,订单时间间隔均值（天）,订单时间间隔方差（天）,交易总量,交易量最大值,交易量最小值,交易量方差,...,交易额方差,交易额均值,交易稳定性（近三个月交易量变化均值）,供应商数量,采购产品种类数量,账期订单占比,账期订单交易量均值,账期订单交易额均值,账期合作时间（月）,最近一笔账期订单（X月之前）
无锡国友塑料制品有限公司,7.0,4.0,1.0,4.0,17.600000,8.384510,170.00,32.00,5.00,13.174651,...,111300.534868,208424.285714,1.518750,3.0,2.0,0.714286,32.000000,273504.000000,4.0,1.0
砀山县雁发塑胶制品有限公司,10.0,16.0,2.0,5.0,48.555556,82.725617,326.14,35.00,30.25,1.722506,...,17273.576581,326036.250000,0.002367,2.0,1.0,1.000000,32.614000,326036.250000,16.0,2.0
宜兴市海燕塑料制品有限公司,23.0,14.0,3.0,1.0,22.666667,16.298683,687.90,47.90,10.00,8.535035,...,65433.125093,272468.316000,0.000000,7.0,1.0,0.869565,34.395000,272468.316000,14.0,3.0
佛山市南海区狮山联生塑料五金厂,20.0,10.0,0.0,4.0,17.117647,30.489511,603.75,50.00,20.00,10.088805,...,90943.561081,279952.236842,0.039636,7.0,2.0,0.400000,28.187500,245304.375000,3.0,0.0
深圳市南璋颜料有限公司,15.0,14.0,6.0,1.0,41.222222,21.683967,403.00,62.00,16.00,16.928772,...,100622.607986,205121.666667,-0.028302,5.0,1.0,0.466667,41.857143,261311.428571,11.0,8.0
佛山市南海辉煌塑料有限公司,17.0,9.0,4.0,5.0,16.125000,12.924782,299.50,40.00,10.00,9.873464,...,79583.805832,155594.117647,0.313234,7.0,3.0,1.000000,17.617647,155594.117647,9.0,4.0
中山市新展塑料制品有限公司,30.0,18.0,1.0,4.0,19.259259,16.225793,1601.25,196.00,28.00,38.805935,...,255532.598634,334245.477241,0.155568,6.0,1.0,0.900000,55.148148,337569.771852,16.0,1.0
四会市塑利莱科技有限公司,25.0,13.0,4.0,1.0,15.909091,18.142056,1463.00,155.00,2.00,29.649245,...,200338.746662,383056.600000,13.714286,6.0,1.0,0.880000,57.954545,375151.590909,12.0,7.0
中山市威塑建材有限公司,2.0,12.0,16.0,1.0,325.000000,0.000000,20.00,10.00,10.00,0.000000,...,8273.149340,61650.000000,NaN,2.0,1.0,1.000000,10.000000,61650.000000,12.0,16.0
惠州俊儿塑料科技有限公司,13.0,10.0,3.0,1.0,33.000000,16.886174,384.69,50.00,24.75,10.051343,...,102676.400125,376344.680000,-0.075595,8.0,2.0,0.769231,38.469000,376344.680000,10.0,3.0


In [19]:
data_add['采购频率（订单总数/合作月数）'] =data_add['订单总数']*1.0/data_add['合作时间（月）']
data_add['采购聚集程度（订单总数/供应商数）'] =data_add['订单总数']*1.0/data_add['供应商数量']
data_add.to_csv('./data/feature_add.csv')
data_add.columns

Index([u'订单总数', u'合作时间（月）', u'最近交易（X月之前）', u'最近连续交易月数', u'订单时间间隔均值（天）',
       u'订单时间间隔方差（天）', u'交易总量', u'交易量最大值', u'交易量最小值', u'交易量方差', u'交易量均值',
       u'交易额总量', u'交易额最大值', u'交易额最小值', u'交易额方差', u'交易额均值',
       u'交易稳定性（近三个月交易量变化均值）', u'供应商数量', u'采购产品种类数量', u'账期订单占比', u'账期订单交易量均值',
       u'账期订单交易额均值', u'账期合作时间（月）', u'最近一笔账期订单（X月之前）', u'采购频率（订单总数/合作月数）',
       u'采购聚集程度（订单总数/供应商数）'],
      dtype='object')

### 客户逾期记录统计
- 逾期统计，占比
- 提取还款数量，占比
- 打标签，以逾期大于16天为界

In [8]:
#所有逾期记录
files = os.listdir('./data/loan_list/')
loan_all = pd.DataFrame()
for f in files:
    record = pd.read_excel(os.path.join('./data/loan_list/',f))
    loan_all = loan_all.append(record.iloc[1:,:])
loan_all.shape    
loan_all['zq_day'] = loan_all[u'账期'].apply(lambda x:re.findall('\d+',x)[0])
loan_all['yq_day'] = loan_all[u'逾期'].apply(lambda x:re.findall('^(-\d+|\d+)',x)[0])
loan_all['zq_day'] = loan_all['zq_day'].astype(int)
loan_all['yq_day'] = loan_all['yq_day'].apply(lambda x:int(x))
loan_all.to_csv('./data/tmp_loan_all.csv',index=None)
loan_all=loan_all[loan_all[u'收款状态']==u'已收款']
loan_all.shape,loan_all.head()

((1391, 15),
    业务类型                客户      已收金额   应收  应收余额              应收单号      应收总额  \
 1  账期业务  佛山市南海区松岗明珠塑料有限公司  184520.0  NaN   0.0  YS20170228152309  184520.0   
 2  账期业务  佛山市南海区松岗明珠塑料有限公司  171990.0  NaN   0.0  YS20170103150373  171990.0   
 3  账期业务  佛山市南海区松岗明珠塑料有限公司  198450.0  NaN   0.0  YS20161117147341  198450.0   
 4  账期业务  佛山市南海区松岗明珠塑料有限公司  198450.0  NaN   0.0  YS20161115147142  198450.0   
 5  账期业务  佛山市南海区松岗明珠塑料有限公司  186940.0  NaN   0.0  YS20161025145675  186940.0   
 
         应收日期 收款状态   账期       起算日期   逾期                 销售合同号  zq_day  yq_day  
 1 2017-03-30  已收款  30天 2017-02-28  11天  ZSL20170227LL-C003WJ      30      11  
 2 2017-01-13  已收款  10天 2017-01-03   0天  ZSL20161228LL-C041WJ      10       0  
 3 2016-12-02  已收款  15天 2016-11-17  -2天  ZSL20161025LL-C009WJ      15      -2  
 4 2016-11-29  已收款  15天 2016-11-14   0天  ZSL20161025LL-C009WJ      15       0  
 5 2016-11-09  已收款  15天 2016-10-25   0天  ZSL20161011LL-C038WJ      15       0  )

In [9]:
def yq_sts(list):
    yq_4=0
    yq_10=0
    yq_15=0
    yq_30=0
    yq_b =0
    zq_num = len(list)
    ad_d=0
    for i in list:
        if i <0:
            ad_d+=1
        elif i>0 and i<=4:
            yq_4+=1
        elif i>4 and i<=10:
            yq_10+=1
        elif i>10 and i<=15:
            yq_15+=1
        elif i>15 and i<=30:
            yq_30+=1
        elif i>30:
            yq_b+=1
            
    return yq_4,yq_10,yq_15,yq_30,yq_b,ad_d,zq_num        

stats = pd.DataFrame()
tmps = loan_all.groupby(by=u'客户')['yq_day'].apply(yq_sts)
stats['4天以内']=tmps.apply(lambda x:x[0])
stats['5-10天']=tmps.apply(lambda x:x[1])
stats['11-15天']=tmps.apply(lambda x:x[2])
stats['16-30天']=tmps.apply(lambda x:x[3])
stats['30天以上']=tmps.apply(lambda x:x[4])
stats['逾期合计'] = stats['4天以内']+stats['5-10天']+stats['11-15天']+stats['16-30天']+stats['30天以上']
stats['提前还款']=tmps.apply(lambda x:x[5])
stats['实际应收笔数']=tmps.apply(lambda x:x[6])
stats['逾期占比']=1.0*stats['逾期合计']/stats['实际应收笔数']
stats['提前还款占比']=1.0*stats['提前还款']/stats['实际应收笔数']
stats['平均账期天数']=loan_all.groupby(by=u'客户')['zq_day'].mean()
stats['平均逾期天数']=loan_all[loan_all['yq_day']>0].groupby(by=u'客户')['yq_day'].mean()
#标签
stats['label_day']=stats['16-30天']+stats['30天以上']
stats['label'] = stats['label_day'].apply(lambda x:1 if x>0 else 0)
stats.to_csv('./data/loan_statistic.csv')
stats.head()

,4天以内,5-10天,11-15天,16-30天,30天以上,逾期合计,提前还款,实际应收笔数,逾期占比,提前还款占比,平均账期天数,平均逾期天数,label_day,label
客户,,,,,,,,,,,,,,
东莞市丰年塑胶制品有限公司,2,0,0,0,0,2,2,8,0.250000,0.250000,7.000000,1.500000,0,0
东莞市佳成塑胶科技有限公司,0,0,0,0,0,0,6,7,0.000000,0.857143,6.857143,NaN,0,0
东莞市喜星塑胶泡棉制品有限公司,1,0,0,0,0,1,3,6,0.166667,0.500000,30.000000,1.000000,0,0
东莞市塑兴塑胶制品有限公司,10,3,0,0,0,13,4,30,0.433333,0.133333,9.966667,3.384615,0,0
东莞市广和新材料有限公司,3,1,3,1,0,8,0,12,0.666667,0.000000,27.750000,9.250000,1,1


In [81]:
#合并所有特征
tmp1 = pd.read_csv('./data/features_card.csv',index_col=0)
tmp2 = pd.read_csv('./data/feature_add.csv',index_col=0)
data_all = tmp1.merge(tmp2,left_index=True,right_index=True,how='left')
stats.index =[x.encode('utf-8') for x in stats.index]
data_all =data_all.merge(stats[['平均逾期天数','平均账期天数','逾期占比','提前还款占比']],left_index=True,right_index=True,how='left')
data_all.to_csv('./data/feature_all.csv')
data_all.shape,data_all.head()

((66, 55),
                  注册地址  注册资本变更  法人变更  分支机构  商标专利  场地归属  有无贷款  企业征信  涉诉信息  \
 无锡国友塑料制品有限公司        1       1     0     0   0.0     0   0.0   NaN     0   
 砀山县雁发塑胶制品有限公司       1       1     0     0   1.0     1   1.0   1.0     0   
 宜兴市海燕塑料制品有限公司       1       0     0     0   1.0     0   1.0   1.0     0   
 佛山市南海区狮山联生塑料五金厂     0       1     0     0   1.0     0   1.0   1.0     0   
 深圳市南璋颜料有限公司         1       0     0     0   0.0     0   1.0   1.0     0   
 
                  下游客户情况    ...     账期订单交易量均值      账期订单交易额均值  账期合作时间（月）  \
 无锡国友塑料制品有限公司        2.0    ...     32.000000  273504.000000        4.0   
 砀山县雁发塑胶制品有限公司       2.0    ...     32.614000  326036.250000       16.0   
 宜兴市海燕塑料制品有限公司       4.0    ...     34.395000  272468.316000       14.0   
 佛山市南海区狮山联生塑料五金厂     2.0    ...     28.187500  245304.375000        3.0   
 深圳市南璋颜料有限公司         2.0    ...     41.857143  261311.428571       11.0   
 
                  最近一笔账期订单（X月之前）  采购频率（订单总数/合作月数）  采购聚集程度（订单总数/供应商数）    平均逾期天数 

### 所有订单数据统计

In [3]:
#所有交易记录
files = VisitDir('./data/client_data/')
files =[f for f in files if '交易记录' in f]
trade = pd.DataFrame()
for f in files:
    t = pd.read_excel(f)
    t =t.dropna(thresh=5)
    trade = trade.append(t)
trade.shape,trade    

((1301, 8),
                       合同编号           客户名称 是否账期 期货/现货  \
 0    ZSL20170703ZWB-E075WJ   无锡国友塑料制品有限公司    是    现货   
 1     ZSL20170612ZWB-E035J   无锡国友塑料制品有限公司    是    现货   
 2     ZSL20170515ZWB-E012W   无锡国友塑料制品有限公司    否    现货   
 3    ZSLB20170515ZWB-E011J   无锡国友塑料制品有限公司    是    现货   
 4    ZSLB20170424ZWB-E009J   无锡国友塑料制品有限公司    是    现货   
 5    ZSLB20170414ZWB-E026W   无锡国友塑料制品有限公司    否    现货   
 6    ZSLB20170406ZWB-E024J   无锡国友塑料制品有限公司    是    现货   
 0    ZSL20170605YLP-E029WJ  砀山县雁发塑胶制品有限公司    是    现货   
 1   ZSLB20170512YLP-E036WJ  砀山县雁发塑胶制品有限公司    是    现货   
 2   ZSLB20170424YLP-E019WJ  砀山县雁发塑胶制品有限公司    是    现货   
 3   ZSLB20170407YLP-E053WJ  砀山县雁发塑胶制品有限公司    是    现货   
 4   ZSLB20170324YLP-E002WJ  砀山县雁发塑胶制品有限公司    是    现货   
 5   ZSLB20170308YLP-E025WJ  砀山县雁发塑胶制品有限公司    是    现货   
 6   ZSLB20170222YLP-E006WJ  砀山县雁发塑胶制品有限公司    是    现货   
 7   ZSLB20160530WXL-E075WJ  砀山县雁发塑胶制品有限公司    是    现货   
 8   ZSLB20160505WXL-E176WJ  砀山县雁发塑胶制品有限公司    是    现货   
 9   ZSLB20160325WX

In [4]:
#所有账期记录
zq_files = os.listdir('./data/loan_list/')
record =pd.DataFrame()
for f in zq_files:
    r = pd.read_excel(os.path.join('./data/loan_list/',f))
    record = record.append(r.iloc[1:,:])  #账期数据
record.shape,record

((1430, 13),
     业务类型                客户       已收金额   应收  应收余额              应收单号       应收总额  \
 1   账期业务  佛山市南海区松岗明珠塑料有限公司  184520.00  NaN   0.0  YS20170228152309  184520.00   
 2   账期业务  佛山市南海区松岗明珠塑料有限公司  171990.00  NaN   0.0  YS20170103150373  171990.00   
 3   账期业务  佛山市南海区松岗明珠塑料有限公司  198450.00  NaN   0.0  YS20161117147341  198450.00   
 4   账期业务  佛山市南海区松岗明珠塑料有限公司  198450.00  NaN   0.0  YS20161115147142  198450.00   
 5   账期业务  佛山市南海区松岗明珠塑料有限公司  186940.00  NaN   0.0  YS20161025145675  186940.00   
 6   账期业务  佛山市南海区松岗明珠塑料有限公司  188500.00  NaN   0.0  YS20161019110425  188500.00   
 7   账期业务  佛山市南海区松岗明珠塑料有限公司  174500.00  NaN   0.0  YS20160926108844  174500.00   
 8   账期业务  佛山市南海区松岗明珠塑料有限公司  174500.00  NaN   0.0  YS20160926108834  174500.00   
 9   账期业务  佛山市南海区松岗明珠塑料有限公司  188790.00  NaN   0.0  YS20160824106578  188790.00   
 10  账期业务  佛山市南海区松岗明珠塑料有限公司  181350.00  NaN   0.0  YS20160811171089  181350.00   
 11  账期业务  佛山市南海区松岗明珠塑料有限公司  115800.00  NaN   0.0   YS2016080317738  115800.00   
 12

In [6]:
record[u'起算日期'].apply(lambda x:str(x.year)+'_'+str(x.month)).value_counts()

2016_9     121
2016_8     118
2017_3     110
2016_10    106
2016_7      98
2016_12     96
2017_2      93
2016_11     92
2016_5      82
2016_6      78
2016_4      76
2016_3      69
2017_6      63
2017_4      62
2017_7      59
2017_5      58
2017_1      47
2017_8       2
Name: 起算日期, dtype: int64

In [15]:
loan_all[loan_all['yq_day']>0][u'起算日期'].apply(lambda x:str(x.year)+'_'+str(x.month)).value_counts()

2016_9     52
2016_10    50
2016_5     49
2016_12    48
2017_3     45
2016_7     41
2016_6     35
2016_8     35
2016_11    34
2017_6     29
2017_2     28
2017_5     26
2016_4     25
2016_3     25
2017_4     19
2017_1     16
2017_7     11
Name: 起算日期, dtype: int64

In [7]:
trade[u'创建时间'].apply(lambda x:str(x.year)+'_'+str(x.month)).value_counts()

2016_10    103
2016_9      96
2016_8      85
2016_3      80
2017_3      79
2016_11     78
2016_12     75
2017_2      72
2016_5      71
2016_6      69
2016_7      67
2017_4      58
2016_4      58
2017_6      56
2017_1      54
2017_5      53
2017_7      44
2016_1      31
2015_12     29
2016_2      15
2015_11     14
2015_9       4
2015_10      4
2015_8       3
2015_7       2
2017_8       1
Name: 创建时间, dtype: int64